# Census APIs

Request a key at <https://api.census.gov/data/key_signup.html>

An email will be sent to the addres you provide. The email will include a link to activate the key. It may take a few minutes before the key can be activated.

In [1]:
# Load API key

# Note the file with the API key is not in the GitHub repo.
# Keys need to be read from a local directory
with open("jwCensusApi", "r") as i:
    key = i.read()

In [2]:
import json
import requests
import pandas as pd
import matplotlib as plt

In [3]:
'''
Current Population Survey Annual Social and Economic Supplement
https://api.census.gov/data.html

Vars: https://api.census.gov/data/2021/cps/asec/mar/variables.html
'''

endpoint = "http://api.census.gov/data/2021/cps/asec/mar" # state geographies
ex1_url = "https://api.census.gov/data/2021/cps/asec/mar?get=NOW_COV,MARSUPWT,A_MARITL&for=state:35&A_HGA=39"
ex2_url = "https://api.census.gov/data/2021/cps/asec/mar?tabulate=weight(MARSUPWT)&col+NOW_COV&row+A_MARITL&for=state:35&A_HGA=39"

In [4]:
api_request_url = ex1_url + "&key=" + key
api_request = requests.get(api_request_url)

In [5]:
api_json = api_request.json()

In [6]:
# print(json.dumps(api_json, indent=4)) # long output

## Identifying the Data Source

We are going to use the Census API to retrieve data to reproduce some of the statistics from the Feb 24, 2022, press release and infographic *A Higher Degree.*

**Press release:** <https://www.census.gov/newsroom/press-releases/2022/educational-attainment.html>

**Infographic:** <https://www.census.gov/library/visualizations/2022/comm/a-higher-degree.html>

Below is the infographic. Note that for today's workshop we are using only 2021 data.

<a class="embeddable-image__embedLink" href="https://www.census.gov/library/visualizations/2022/comm/a-higher-degree.html?cid=higher-degree" target="_blank"><img class="embeddable-image__embedImage" data-src="/library/visualizations/2022/comm/a-higher-degree/_jcr_content/root/responsivegrid/embeddableimage1119.coreimg.jpeg/1645624284211/educational-attainment-2022.jpeg" alt="A Higher Degree" title="A Higher Degree" src="https://www.census.gov/library/visualizations/2022/comm/a-higher-degree/_jcr_content/root/responsivegrid/embeddableimage1119.coreimg.jpeg/1645624284211/educational-attainment-2022.jpeg" width="1080" height="1080"></a>[Source: U.S. Census Bureau]

According to the press release, the data are from the *2000–2002 March Current Population Survey and 2003–2021 Annual Social and Economic Supplement to the Current Population Survey.*

### Q: How to get info on variables via API?

We can look it up online, but we may also get this info dynamically. 

#### Steps:

1. Go to the *Census Data API Discovery Tool*: <https://www.census.gov/data/developers/updates/new-discovery-tool.html>
1. Preview the datasets and available information using the HTML format: <https://api.census.gov/data.html>
1. Using a process similar to the above, we can also retrieve the information in JSON format.

In [51]:
# Request the JSON information about available datasets

datasets_url = "https://api.census.gov/data.json"
datasets_request = requests.get(datasets_url)
datasets_json = datasets_request.json()

In [63]:
# this took a little sleuthing...

# Get the titles of the datasets
dataset_list = datasets_json["dataset"]

# title: Current Population Survey Annual Social and Economic Supplement
# c_vintage: 2021
# id: https://api.census.gov/data/id/CPSASEC2021

cpd_asec_mar = [d for d in dataset_list if d["identifier"] == "https://api.census.gov/data/id/CPSASEC2021"]
print(json.dumps(cpd_asec_mar, indent=4)) # long output

[
    {
        "c_vintage": 2021,
        "c_dataset": [
            "cps",
            "asec",
            "mar"
        ],
        "c_geographyLink": "http://api.census.gov/data/2021/cps/asec/mar/geography.json",
        "c_variablesLink": "http://api.census.gov/data/2021/cps/asec/mar/variables.json",
        "c_tagsLink": "http://api.census.gov/data/2021/cps/asec/mar/tags.json",
        "c_examplesLink": "http://api.census.gov/data/2021/cps/asec/mar/examples.json",
        "c_groupsLink": "http://api.census.gov/data/2021/cps/asec/mar/groups.json",
        "c_sorts_url": "http://api.census.gov/data/2021/cps/asec/mar/sorts.json",
        "c_valuesLink": "http://api.census.gov/data/2021/cps/asec/mar/values.json",
        "c_documentationLink": "https://www.census.gov/developer/",
        "c_isMicrodata": true,
        "c_isCube": true,
        "c_isAvailable": true,
        "@type": "dcat:Dataset",
        "title": "Current Population Survey Annual Social and Economic Supplement",
   

In [74]:
# Get info on variables

dataset_vars_url = cpd_asec_mar[0]["c_variablesLink"]
dataset_vars_request = requests.get(dataset_vars_url)
datasets_vars_json = dataset_vars_request.json()
# print(json.dumps(datasets_vars_json, indent=4)) # long output - many variables
# just print the variable and the label
dataset_vars = datasets_vars_json["variables"]
# sort variable names
sorted_vars = sorted(dataset_vars.keys())
for k in sorted_vars[:10]:
    var_info = dataset_vars[k]
    print(k, ":", var_info["label"])

ACTC_CRD : additional chld tax credit
AGE1 : Demographics, Age recode, persons 15+ years
AGI : Federal adjusted gross income
ANN_VAL : Retirement income, annuities amount
ANN_YN : Retirement income, annuities, y/n
AXAGE : Allocation flag for A_AGE
AXCLSWKR : Allocation flag for A_CLSWKR
AXENRLW : Allocation flag for A_ENRLW
AXFTPT : Allocation flag for A_FTPT
AXHGA : Allocation flag for A_HGA


In [75]:
# Maybe create dictionaries of vars:defs
# populate via iteration

var = "NOW_COV"
var_def_url = endpoint + "/variables/" + var

In [76]:
var_def_request = requests.get(var_def_url)

In [77]:
var_json = var_def_request.json()

In [78]:
print(json.dumps(var_json, indent=4))

{
    "name": "NOW_COV",
    "label": "Currently covered by health insurance coverage",
    "predicateType": "int",
    "group": "N/A",
    "limit": 0,
    "suggested-weight": "MARSUPWT",
    "values": {
        "item": {
            "2": "No",
            "1": "Yes"
        }
    }
}


In [79]:
'''
Variables of interest

A_HGA - educational attainment
AGE1 - age
AGI - federal adjusted gross income

NM FIPS ID = 35
'''

vars_list = ["A_HGA", "AGE1", "AGI"]
vars_dict = {}

for v in vars_list:
    var_def_url = endpoint + "/variables/" + v
    var_def_request = requests.get(var_def_url)
    var_json = var_def_request.json()
    vars_dict[v] = var_json

In [80]:
vars_dict

{'A_HGA': {'name': 'A_HGA',
  'label': 'Demographics, Educational attainment',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  'values': {'item': {'35': '9th Grade',
    '41': 'Assc degree-occupation/vocation',
    '0': 'Children',
    '31': 'Less Than 1st Grade',
    '44': "Master's degree (MA,MS,MENG,MED,MSW,MBA)",
    '43': "Bachelor's degree (BA,AB,BS)",
    '34': '7th and 8th grade',
    '38': '12th Grade No Diploma',
    '37': '11th Grade',
    '33': '5th Or 6th Grade',
    '45': 'Professional school degree (MD,DDS,DVM,L',
    '39': 'High school graduate-high school diploma',
    '32': '1st,2nd,3rd,or 4th grade',
    '40': 'Some College But No Degree',
    '46': 'Doctorate degree (PHD,EDD)',
    '36': '10th Grade',
    '42': 'Assc degree-academic program'}}},
 'AGE1': {'name': 'AGE1',
  'label': 'Demographics, Age recode, persons 15+ years',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  

In [13]:
# API expects vars to be comma separated, no spaces

vars_query = ','.join(str(v) for v in vars_list)
print(vars_query)

A_HGA,AGE1,AGI


In [14]:
# request non-tabulated data
# URL format: "https://api.census.gov/data/2021/cps/asec/mar?get=NOW_COV,MARSUPWT,A_MARITL&for=state:35"

# We can parameterize the state but for now we will hard code all and NM (35)
api_request_url = endpoint + "?get=" + vars_query + "&for=state:*&key=" + key
api_request = requests.get(api_request_url)
api_json = api_request.json()

In [15]:
type(api_json) # results seem to generally be lists, but check in case a dictionary is returned

list

In [16]:
census_df = pd.read_json(json.dumps(api_json), typ = "frame", orient = "values")

In [17]:
census_df.head()

,0,1,2,3
0,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23


In [18]:
census_df.columns = census_df.iloc[0]

In [19]:
census_df.head()

,A_HGA,AGE1,AGI,state
0,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23


In [20]:
census_df = census_df.drop(census_df.index[0])

In [21]:
census_df.head()

,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23
5,39,15,0,23


In [22]:
census_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163543 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   A_HGA   163543 non-null  object
 1   AGE1    163543 non-null  object
 2   AGI     163543 non-null  object
 3   state   163543 non-null  object
dtypes: object(4)
memory usage: 6.2+ MB


In [23]:
# We can separately request data for just NM, or we can subset
# recall that the FIPS code for NM is 35

nm_census_data = census_df[census_df["state"] == "35"].copy() # quotes around "35" b/c it is an object not an int

In [24]:
nm_census_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2900 entries, 126038 to 128937
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A_HGA   2900 non-null   object
 1   AGE1    2900 non-null   object
 2   AGI     2900 non-null   object
 3   state   2900 non-null   object
dtypes: object(4)
memory usage: 113.3+ KB


In [25]:
# out of curiosity let's get the NM data via API
# same as before with one different parameter

api_request_url = endpoint + "?get=" + vars_query + "&for=state:35&key=" + key
api_request = requests.get(api_request_url)
api_json = api_request.json()

nm_census_df = pd.read_json(json.dumps(api_json), typ = "frame", orient = "values")
nm_census_df.columns = nm_census_df.iloc[0]
nm_census_df = nm_census_df.drop(nm_census_df.index[0])

In [26]:
nm_census_df.info() # the data have the same number of rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2900 entries, 1 to 2900
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   A_HGA   2900 non-null   object
 1   AGE1    2900 non-null   object
 2   AGI     2900 non-null   object
 3   state   2900 non-null   object
dtypes: object(4)
memory usage: 113.3+ KB


In [27]:
# Let's make a function to get the data and convert to data frame
# "https://api.census.gov/data/2021/cps/asec/mar?tabulate=weight(MARSUPWT)&col+NOW_COV&row+A_MARITL&for=state:35&A_HGA=39"
def get_census_data(endpoint_url, vars_list, key, state = "*"):
    # First, convert vars_list to string
    vars_query = ','.join(str(v) for v in vars_list)
    
    # Build the request URL
    api_request_url = endpoint_url + "?get=" + vars_query + "&for=state:" + state + "&key=" + key
    
    # Request the data
    api_request = requests.get(api_request_url)
    api_json = api_request.json()
    
    # Convert returned JSON to dataframe
    df = pd.read_json(json.dumps(api_json), typ = "frame", orient = "values")
    
    # Use first row values for col headers
    df.columns = df.iloc[0]
    
    # Drop first row, which is still header info
    df = df.drop(df.index[0])
    
    return df

# While we're at it, create a function to get var defs
# No key needed
def get_census_vars(vars_list):
    vars_dict = {}
    for v in vars_list:
        var_def_url = endpoint + "/variables/" + v
        var_def_request = requests.get(var_def_url)
        var_json = var_def_request.json()
        vars_dict[v] = var_json
    return vars_dict

In [28]:
# Now we can make ad hoc requests
# but we will redo our initial request

vars_list = ["A_HGA", "AGE1", "AGI"]
var_info = get_census_vars(vars_list)
census_data = get_census_data(endpoint, vars_list, key) # use default * for all state data

In [29]:
census_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163543 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   A_HGA   163543 non-null  object
 1   AGE1    163543 non-null  object
 2   AGI     163543 non-null  object
 3   state   163543 non-null  object
dtypes: object(4)
memory usage: 6.2+ MB


In [30]:
# try again with just NM
# census_data = get_census_data(endpoint, vars_list, key, state = "35") # NM
# census_data.info()

In [31]:
# Things work - let's see about reproducing stats from the press release
# Right now we have age and education attainment (and income) for 2021

'''
In 2021, the highest level of education of the population age 25 and older in the United States 
was distributed as follows: 

    8.9% had less than a high school diploma or equivalent.
    27.9% had high school graduate as their highest level of school completed. 
    14.9% had completed some college but not a degree.
    10.5% had an associate degree as their highest level of school completed.
    23.5% had a bachelor’s degree as their highest degree.
    14.4% had completed an advanced degree such as a master’s degree, professional degree or doctoral degree. 
'''

var_info

{'A_HGA': {'name': 'A_HGA',
  'label': 'Demographics, Educational attainment',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  'values': {'item': {'35': '9th Grade',
    '41': 'Assc degree-occupation/vocation',
    '0': 'Children',
    '31': 'Less Than 1st Grade',
    '44': "Master's degree (MA,MS,MENG,MED,MSW,MBA)",
    '43': "Bachelor's degree (BA,AB,BS)",
    '34': '7th and 8th grade',
    '38': '12th Grade No Diploma',
    '37': '11th Grade',
    '33': '5th Or 6th Grade',
    '45': 'Professional school degree (MD,DDS,DVM,L',
    '39': 'High school graduate-high school diploma',
    '32': '1st,2nd,3rd,or 4th grade',
    '40': 'Some College But No Degree',
    '46': 'Doctorate degree (PHD,EDD)',
    '36': '10th Grade',
    '42': 'Assc degree-academic program'}}},
 'AGE1': {'name': 'AGE1',
  'label': 'Demographics, Age recode, persons 15+ years',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  

In [32]:
# First subset to AGE1 values >= 6 (25 and older)

# convert cols to int
census_data = census_data.astype(int)
census_data.info()
age_gte25 = census_data[census_data["AGE1"] >= 6].copy()
age_gte25.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 163543 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A_HGA   163543 non-null  int32
 1   AGE1    163543 non-null  int32
 2   AGI     163543 non-null  int32
 3   state   163543 non-null  int32
dtypes: int32(4)
memory usage: 3.7 MB
<class 'pandas.core.frame.DataFrame'>
Int64Index: 108998 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype
---  ------  --------------   -----
 0   A_HGA   108998 non-null  int32
 1   AGE1    108998 non-null  int32
 2   AGI     108998 non-null  int32
 3   state   108998 non-null  int32
dtypes: int32(4)
memory usage: 2.5 MB


In [33]:
age_gte25.head()

,A_HGA,AGE1,AGI,state
1,39,12,10000,23
2,39,12,0,23
3,39,17,0,23
4,43,15,80819,23
5,39,15,0,23


In [34]:
var_info

{'A_HGA': {'name': 'A_HGA',
  'label': 'Demographics, Educational attainment',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  'values': {'item': {'35': '9th Grade',
    '41': 'Assc degree-occupation/vocation',
    '0': 'Children',
    '31': 'Less Than 1st Grade',
    '44': "Master's degree (MA,MS,MENG,MED,MSW,MBA)",
    '43': "Bachelor's degree (BA,AB,BS)",
    '34': '7th and 8th grade',
    '38': '12th Grade No Diploma',
    '37': '11th Grade',
    '33': '5th Or 6th Grade',
    '45': 'Professional school degree (MD,DDS,DVM,L',
    '39': 'High school graduate-high school diploma',
    '32': '1st,2nd,3rd,or 4th grade',
    '40': 'Some College But No Degree',
    '46': 'Doctorate degree (PHD,EDD)',
    '36': '10th Grade',
    '42': 'Assc degree-academic program'}}},
 'AGE1': {'name': 'AGE1',
  'label': 'Demographics, Age recode, persons 15+ years',
  'predicateType': 'int',
  'group': 'N/A',
  'limit': 0,
  'suggested-weight': 'MARSUPWT',
  

In [35]:
# convert A_HGA, AGE1, and state to categories
age_gte25.A_HGA = age_gte25.A_HGA.astype("category")
age_gte25.AGE1 = age_gte25.AGE1.astype("category")
age_gte25.state = age_gte25.state.astype("category")

In [36]:
age_gte25.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 108998 entries, 1 to 163543
Data columns (total 4 columns):
 #   Column  Non-Null Count   Dtype   
---  ------  --------------   -----   
 0   A_HGA   108998 non-null  category
 1   AGE1    108998 non-null  category
 2   AGI     108998 non-null  int32   
 3   state   108998 non-null  category
dtypes: category(3), int32(1)
memory usage: 1.6 MB


In [37]:
type(age_gte25.iloc[0, 0])

numpy.int64

In [38]:
age_gte25.iloc[0, 0]

39

In [39]:
# so cols are categories but cell values are still int (?)

# Educational attainment: <= 38 less than HS diploma or eq
# 39 == HS diploma, 40 == some college ND, 41 & 42 == assoc Deg, 43 == Bachelor's, 
# 44 & 45 Masters or professional degree, 46 == doctorate

# there is probably a faster way to do this
ed_groups = age_gte25.groupby("A_HGA")

In [40]:
#ed_groups.groups

In [41]:
# 27.9% HS grads per press release
hs_grads = ed_groups.get_group(39)

In [42]:
len(hs_grads)

30656

In [43]:
(30656 / 108998) * 100

28.1252867025083

In [44]:
# 14.9% had some college

some_college_nd = ed_groups.get_group(40)

In [45]:
len(some_college_nd)

16369

In [46]:
(16369/108998) * 100

15.017706746912788

In [47]:
# 31-38 for no HS
# 8.9% report less than HS diploma or eq

no_diploma = 0
for i in range(31, 39):
    g = ed_groups.get_group(i)
    no_diploma += len(g)

In [48]:
no_diploma

10289

In [49]:
(10289/108998) * 100

9.439622745371475

In [152]:
# Values are different from press release - try weighted data

census_wtd_url = "https://api.census.gov/data/2021/cps/asec/mar?tabulate=weight(MARSUPWT)&col+A_HGA&row+AGE1&for=state:*"

In [153]:
census_wtd_request = requests.get(census_wtd_url)

In [154]:
census_wtd_json = census_wtd_request.json()

In [155]:
print(json.dumps(census_wtd_json, indent=4)) # long output

[
    [
        {
            "A_HGA": "35"
        },
        {
            "A_HGA": "41"
        },
        {
            "A_HGA": "0"
        },
        {
            "A_HGA": "31"
        },
        {
            "A_HGA": "44"
        },
        {
            "A_HGA": "43"
        },
        {
            "A_HGA": "34"
        },
        {
            "A_HGA": "38"
        },
        {
            "A_HGA": "37"
        },
        {
            "A_HGA": "33"
        },
        {
            "A_HGA": "45"
        },
        {
            "A_HGA": "39"
        },
        {
            "A_HGA": "32"
        },
        {
            "A_HGA": "40"
        },
        {
            "A_HGA": "46"
        },
        {
            "A_HGA": "36"
        },
        {
            "A_HGA": "42"
        },
        "AGE1"
    ],
    [
        314869,
        914820,
        0,
        34124,
        2658149,
        5036686,
        298526,
        283375,
        282743,
        307977,
        39

In [198]:
# Convert returned JSON to dataframe
drop_keys = census_wtd_json[1:]
get_keys = census_wtd_json[0]
w_df = pd.read_json(json.dumps(drop_keys), typ = "frame", orient = "values")
cols = []
for k in get_keys:
    if isinstance(k, dict):
        v = [e[:] for e in k.values()]
        cols.append(v[0])
    if isinstance(k, str):
        cols.append(k)
w_df.columns = cols
w_df.set_index("AGE1", inplace = True)
print(w_df.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 18 entries, 9 to 17
Data columns (total 17 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   35      18 non-null     int64
 1   41      18 non-null     int64
 2   0       18 non-null     int64
 3   31      18 non-null     int64
 4   44      18 non-null     int64
 5   43      18 non-null     int64
 6   34      18 non-null     int64
 7   38      18 non-null     int64
 8   37      18 non-null     int64
 9   33      18 non-null     int64
 10  45      18 non-null     int64
 11  39      18 non-null     int64
 12  32      18 non-null     int64
 13  40      18 non-null     int64
 14  46      18 non-null     int64
 15  36      18 non-null     int64
 16  42      18 non-null     int64
dtypes: int64(17)
memory usage: 2.5 KB
None


In [199]:
w_df.head()

,35,41,0,31,44,43,34,38,37,33,45,39,32,40,46,36,42
AGE1,,,,,,,,,,,,,,,,,
9,314869,914820,0,34124,2658149,5036686,298526,283375,282743,307977,394458,5039375,112508,2743662,468306,269929,1237130
6,186793,903353,0,35343,1752186,6668554,127343,285213,354979,91872,200201,6304709,23674,3891482,185560,175782,1405260
5,92577,389627,0,15990,211669,3181597,67912,177681,233548,43131,10135,3721697,11315,3494541,7623,139139,744376
4,51054,171027,0,15589,16770,222168,57125,202790,206899,7866,0,2919202,6153,4163847,0,85361,353451
8,310629,876556,0,47813,2714555,5746319,279523,270560,300046,277076,379479,5002109,99229,3083392,544623,292238,1383291
